In [267]:
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from os.path import expanduser
import os.path 
import numpy as np
import webbrowser as wb

#TODO:
# create helper functions: data quality binning
#add liftoff and start of takeoff points and quality, computation of takeoff roll length
#		○ Delineate/mark the whole ground roll event (as detected) in the trajectory dataframe
#			§ Helper function to get ground roll start time
#			§ Helper function to get ground roll end time
#		○ Store ground roll distance in the schedule table
#		○ Store N1_during_takeoff_groundroll in the schedule table
#           Numerical Average during ground roll event

home = expanduser("~")
filepath = home+'\OneDrive - DOT OST\BADA4_Reduced_Thrust_Sensor_Path_Noise_Comparison_main'#\\FDR_desensitized\\TKOFF'
filename = 'takeoff_distance'#_kluge'
file_extension = '.csv'
df = pd.read_csv(os.path.join(filepath, filename + file_extension), index_col=["FLIGHT_ID", "TIME_OFFSET"])
df.rename(columns={'FUEL_QUANTITY':'FUEL_QUANTITY_KILOGRAMS'}, inplace=True)


In [268]:
# #optionally write the desensitized file version if you like
# df_desensitized = pd.read_csv(os.path.join(filepath+'\\FDR_desensitized\\TKOFF', filename+'_desensitized'+file_extension), index_col=["FLIGHT_ID", "TIME_OFFSET"])
# df_kluge = pd.concat([df_desensitized,df[["FUEL_QUANTITY_KILOGRAMS"]]], axis = 1)
# df_kluge.APT_encoded = df_kluge.APT_encoded.astype('str')
# df_kluge.Departure_Runway_end_encoded = df_kluge.Departure_Runway_end_encoded.astype('str')
# df_kluge.STAGE_LENGTH_ID = df_kluge.STAGE_LENGTH_ID.astype('str')
# df_grouping_series = df_kluge[["APT_encoded", "ACTYPE", "Departure_Runway_end_encoded", "STAGE_LENGTH_ID"]].dropna()
# df_kluge["APT_AIRCRAFT_RUNWAY_STAGE.1"] = df_grouping_series.apply(list, axis=1).str.join(sep='_')
# df_kluge.to_csv(path_or_buf=filepath+'\\FDR_desensitized\\TKOFF\\takeoff_distance_kluge.csv')

In [269]:
modes = ['Undetected','Takeoff Ground Roll']

df["Detected_Trajectory_Mode"] = modes.index('Undetected')
df.Detected_Trajectory_Mode = df.Detected_Trajectory_Mode.astype("int")

In [270]:
df_liftoff = pd.DataFrame(df[df.DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF.notna()])
df_liftoff.reset_index(level = 1, inplace = True)

df_rollstart = df[df.DISTANCE_FROM_RUNWAY_END_AT_DETECTED_START_OF_TAKEOFF.notna()]
df_rollstart.reset_index(level = 1, inplace = True)

In [271]:
df_rollstart.loc[:,"APT_AIRCRAFT_RUNWAY_STAGE.1"]

FLIGHT_ID
1001880    NaN
1002404    NaN
1003312    NaN
1003324    NaN
1003783    NaN
          ... 
1154656    NaN
1154938    NaN
1155338    NaN
1156104    NaN
1156385    NaN
Name: APT_AIRCRAFT_RUNWAY_STAGE.1, Length: 247, dtype: object

In [285]:
def plot_metrics_for_individual_flights(dataframe, flight_sample_size = 10):

    metric_maxes = {}

    plotbook_filename = 'plotbook' + '_' + str(flight_sample_size) if flight_sample_size != -1 else 'plotbook'
    df = dataframe
    if 'FLIGHT_ID' in df.columns:
        df_grouped_by_flight = df.drop(columns = ['FLIGHT_ID'], axis = 1).groupby("FLIGHT_ID")
        df["FLIGHT_ID"] = df["FLIGHT_ID"].astype(str)
    else:
        df_grouped_by_flight = df.groupby("FLIGHT_ID")
    
    fn = plotbook_filename + '.html'
    open( os.path.join(filepath,fn), 'w')
    #     #TODO: write plots to pdf file
    #     fn = plotbook_filename + '.pdf'

    flight_count = 0
    for flight, group in df_grouped_by_flight:
        flight_sample_size = len(group) if flight_sample_size == -1 else flight_sample_size

        if flight_count == flight_sample_size:
            break

#         metrics = ['MSL_ALT']
        metrics = ['AFE_ALT']
        metrics += ['N1', 'TAS_SEGMENT']

        #create Figure with subplots for each metric for a given flight
        fig = make_subplots(rows=len(metrics), cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.02)

        for m in metrics:
            metric_maxes[m]=group[m].max() if m != 'N1' else 100

            #create and add a subplot to the Figure
            sub = go.Scatter(
                    x=group.DISTANCE_FROM_RUNWAY_END
                    , y=group[m]
                    , mode="markers"
                    , name = m
            )
            fig.add_trace(sub, row=metrics.index(m)+1, col=1)

    #         #turn off auto range adjustment
    #         fig.update_layout(
    #             yaxis_autorange = False
    #             )        
    #         fig.update_yaxes({}, row=metrics.index(m)+1, col=1)

            dl_cgtd = detected_liftoff_cumul_grnd_trk_dist = df_liftoff.loc[flight,"DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF"]
            fig.add_trace(
                go.Scatter(
                    x=[dl_cgtd, dl_cgtd]
                    , y=[0,metric_maxes[m]]
                    , mode = "lines"
                    , name = "Detected Liftoff"
                    , line = go.scatter.Line(color = 'gray')
                )
                , row=metrics.index(m)+1, col=1 
            )        

            rprt_liftoff_cgtd = reported_liftoff_cumul_grnd_trk_dist = df_liftoff.loc[flight,"DISTANCE_FROM_RUNWAY_END"]
            fig.add_trace(
                go.Scatter(
                    x=[rprt_liftoff_cgtd, rprt_liftoff_cgtd]
                    , y=[0,metric_maxes[m]]
                    , mode = "lines"
                    , name = "Reported Liftoff"
                    , line = go.scatter.Line(color = 'orange', dash = 'dash')
                )
                , row=metrics.index(m)+1, col=1 
            )

            fig.layout.yaxis.update(title_text = metrics[0])
            fig.layout.yaxis2.update(title_text = metrics[1], tickvals=[0] + np.arange(70,100,5))
            fig.layout.yaxis3.update(title_text = metrics[2])
            fig.layout.update(title_text = "Flight " + str(flight) + '<br>' + 'Airport / Aicraft Type / Runway / Stage Length ID: ' + 
                              str(df_liftoff.loc[flight,"APT_AIRCRAFT_RUNWAY_STAGE.1"]))
        
        #write plots to html file         
        html = fig.to_html()
        with open( os.path.join(filepath,fn), 'a') as f:
            f.write(html) 

        flight_count = flight_count + 1
    print(os.path.join(filepath,fn))
        
    return (os.path.join(filepath,fn))

In [273]:
salient_rollstart_col_list = ['DISTANCE_FROM_THRESHOLD_AT_POINT_ONE', 'DISTANCE_FROM_POINT_ONE_FEET',
       'GROUNDSPEED_AT_POINT_ONE_KNOTS', 'WELL-BEHAVED_TRAJECTORY',
#        'LATITUDE_AT_POINT_ONE', 'LONGITUDE_AT_POINT_ONE',
        'STATED_SEGMENT_START_OF_TAKEOFF',
       'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_START_OF_TAKEOFF',
        'APT_AIRCRAFT_RUNWAY_STAGE.1']
extraneous_schedule_columns_list = [col for col in df_rollstart.columns if col not in salient_rollstart_col_list + ["DISTANCE_FROM_RUNWAY_END", 'TIME_OFFSET','FUEL_QUANTITY_KILOGRAMS', 'HEAD_WIND']]



In [274]:
df_schedule = df_liftoff.drop(salient_rollstart_col_list, axis = 1).join(
    df_rollstart.drop(
        columns = extraneous_schedule_columns_list, axis = 1
    )
    , how = 'left'
    , lsuffix = '_liftoff' 
    , rsuffix = '_rollstart'
)

df_schedule["TAKEOFF_GROUND_ROLL_DISTANCE_DETECTED_IN_FEET"] = df_schedule.DISTANCE_FROM_RUNWAY_END_liftoff - df_schedule.DISTANCE_FROM_RUNWAY_END_rollstart

 len(df[df["WELL-BEHAVED_TRAJECTORY"] == 1]) rows in the sample are well-behaved

In [275]:
df_schedule["liftoff_detection_quality"] = abs(df_liftoff.DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF - df_liftoff.DISTANCE_FROM_RUNWAY_END) / df_liftoff.DISTANCE_FROM_RUNWAY_END 
#instantiate bins for grouping on the quality of liftoff detection
liftoff_detection_quality_bins = [0, 0.01] #, 0.05, 0.1, 1]
df_schedule["very_high_liftoff_detection_quality"] = df_schedule["liftoff_detection_quality"].between(liftoff_detection_quality_bins[0], liftoff_detection_quality_bins[1]) == True
flights_with_very_high_liftoff_detection_quality = df_schedule[df_schedule.very_high_liftoff_detection_quality == True].index.to_series()

In [276]:
#def mark_ground_roll_segments(df, flight_id, liftoff_time_offset, rollstart_time_offset, ground_roll_mode_index):
    #return a dataframe of start and stop times by flight
#select rows where TIME_OFFSET is within TIME_OFFSET_rollstart and TIME_OFFSET_liftoff
df_aug = df.loc[:,['N1']]
df_aug = df_aug.join(df_schedule, rsuffix="_sched", how="left").reset_index(level="TIME_OFFSET")
df_aug["Detected_Ground_Roll"] = (df_aug.TIME_OFFSET >= df_aug.TIME_OFFSET_rollstart) & (df_aug.TIME_OFFSET <= df_aug.TIME_OFFSET_liftoff)
df_aug.set_index('TIME_OFFSET',append=True, inplace=True)
mask = (df_aug.Detected_Ground_Roll == True)
df.loc[mask, 'Detected_Trajectory_Mode'] = modes.index('Takeoff Ground Roll')

In [277]:
#add average N1 during detected ground roll to the schedule
df_traj_takeoff_ground_roll = df[df.Detected_Trajectory_Mode == modes.index('Takeoff Ground Roll')]
df_traj_tkoff_gd_rl_groupby = df_traj_takeoff_ground_roll.groupby(by=["FLIGHT_ID"])
df_schedule["N1_during_detected_takeoff_ground_roll"] = df_traj_tkoff_gd_rl_groupby.mean().N1

In [278]:
#plot relationship of actual N1 to actual ground roll distance for a given weight, weather, runway, aircraft type
#plot relationship of actual N1 to detected ground roll distance for a given weight, weather, runway, aircraft type
#plot relationship of actual N1 to detected N1 for a given weight, weather, runway, aircraft type
inp_vars = ["HEAD_WIND_liftoff", "FUEL_QUANTITY_KILOGRAMS_rollstart"]
#placeholder columns to hold categorizations
df_schedule["FUEL_ROLL_BIN"] = pd.Series(dtype='str')
df_schedule["HEADWIND_LFTFF_BIN"] = pd.Series(dtype='str')

#create the categorizations / numerical bins on the input variables in inp_vars list
#weather bins with width of 2 knots
#weight bins with width computed dynamically as 1% of the runway/aircraft type/stage group's max weight
df_schedule_groupby = df_schedule.groupby(by="APT_AIRCRAFT_RUNWAY_STAGE.1")
group_list = list(df_schedule_groupby.indices.keys())


#loop through groups, adding row by row and taking cuts at bins, a column to a dataframe
df_schedule_group = pd.DataFrame(index=group_list, columns=inp_vars)
prev_group = None
frame = pd.DataFrame()
for group, inp_var in df_schedule_groupby:
    #pass the group-inp_var combo as a series extracted from the groupby object
    adding_column_data = True if group == prev_group else False
    axis = 1 if adding_column_data else 0    
    num_bins = 10

    var_group_combo_as_series = df_schedule_groupby[i].get_group(group)
    (var_group_combo_binned, bins) = pd.cut(var_group_combo_as_series
                       , num_bins
                       , labels=False
                       , retbins=True
                      )

    #store the rows or columns
    frame = pd.concat([frame, var_group_combo_binned], axis = axis, verify_integrity=True)

    #store the category definitions
#     df_schedule_group
    
    prev_group = group    

In [279]:
frame.head()

""


In [280]:
df_schedule_groupby["HEAD_WIND_rollstart"]#.get_group("LAX_A340_06R_7")

In [281]:
max_roll_fuel_in_group = df_schedule_groupby.max().FUEL_QUANTITY_KILOGRAMS_rollstart
min_roll_fuel_in_group = df_schedule_groupby.min().FUEL_QUANTITY_KILOGRAMS_rollstart
df_weight_range = pd.DataFrame(min_roll_fuel_in_group).join(max_roll_fuel_in_group, lsuffix='_min', rsuffix='_max')

0.01*max_roll_fuel_in_group



Series([], Name: FUEL_QUANTITY_KILOGRAMS_rollstart, dtype: float64)

In [286]:
df_high_qual_lift = df.join(flights_with_very_high_liftoff_detection_quality,how = 'inner')
fp = plot_metrics_for_individual_flights(df_high_qual_lift, -1)
wb.open(url=fp)

C:\Users\Lyle.Tripp\OneDrive - DOT OST\BADA4_Reduced_Thrust_Sensor_Path_Noise_Comparison_main\plotbook.html


True

In [283]:
#focus in on one pair of flights
#df_schedule = df_schedule.loc[[1124373, 1128101],:]

In [284]:
x_max = stated_max = df_schedule.DISTANCE_FROM_RUNWAY_END_liftoff.max()
x_min = stated_min = df_schedule.DISTANCE_FROM_RUNWAY_END_liftoff.min()
y_max = detected_max = df_schedule.DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF.max()
y_min = detected_min = df_schedule.DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF.min()

perfect_fit_min = x_min if x_min < y_min else y_min
perfect_fit_max = x_max if x_max > y_max else y_max

fig = px.scatter(
    df_schedule.reset_index(level = 0)
    , title='Liftoff distance from runway end (feet)'
    , x='DISTANCE_FROM_RUNWAY_END_liftoff'
    ,y='DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF'
    , color="APT_AIRCRAFT_RUNWAY_STAGE.1" 
    , labels = {'DISTANCE_FROM_RUNWAY_END_liftoff':'As stated in CFDR',
                'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF':'As detected from trajectory',
                'APT_AIRCRAFT_RUNWAY_STAGE.1': 'Airport / Aicraft Type / Runway / Stage Length ID'
               }
    , hover_data= ['FLIGHT_ID']#, 'DISTANCE_FROM_RUNWAY_END', 'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_LIFTOFF']
)

fig.add_trace(
    go.Scatter(
        x = [perfect_fit_min, perfect_fit_max]
        , y = [perfect_fit_min, perfect_fit_max]
        , mode = "lines"
        , name = 'Perfect Fit'
        , line = go.scatter.Line(color = 'black', dash = 'dash')
        , opacity=0.4
    )
)

fig.add_trace(
    go.Scatter(
        x = [perfect_fit_min, perfect_fit_max]
        , y = [perfect_fit_min+275, perfect_fit_max+275]
        , mode = "lines"
        , name = 'Perfect Fit + 275 ft'
        , line = go.scatter.Line(color = 'black', dash = 'longdash')
        , opacity=0.4
    )

)

fig.add_trace(
    go.Scatter(
        x = [perfect_fit_min, perfect_fit_max]
        , y = [perfect_fit_min+550, perfect_fit_max+550]
        , mode = "lines"
        , name = 'Perfect Fit + 550 ft'
        , line = go.scatter.Line(color = 'black', dash = 'longdashdot')
        , opacity=0.4
    )

)

fig.update_yaxes(
#     scaleanchor = "x",
    scaleratio = 1,
  )

fig.show()

file = fig.to_html()
with open(filepath+'\plot_liftoff.html', 'w') as f:
    f.write(file)
    

IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
#two flights with similar stated liftoff point 
flights = [1128101, 1124373]
plot_metrics_for_individual_flights(df.loc[flights,])


In [ ]:
fig = px.scatter(
    df_rollstart
    , title='Start of takeoff roll, distance from runway end (feet)'
    , x='DISTANCE_FROM_RUNWAY_END'
    , y='DISTANCE_FROM_RUNWAY_END_AT_DETECTED_START_OF_TAKEOFF'
    , color="APT_AIRCRAFT_RUNWAY_STAGE.1" 
    , labels = {'DISTANCE_FROM_RUNWAY_END':'As stated in CFDR',
                'DISTANCE_FROM_RUNWAY_END_AT_DETECTED_START_OF_TAKEOFF':'As detected from trajectory',
                'APT_AIRCRAFT_RUNWAY_STAGE.1': 'Airport / Aicraft Type / Runway / Stage Length ID'
               }
)

fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

fig.show()

file = fig.to_html()
with open(filepath+'\plot_rollstart.html', 'w') as f:
    f.write(file)

In [ ]:
fig_px = px.scatter(
    df_liftoff
    , title='Liftoff distance from runway end (feet)'
    , x='DISTANCE_FROM_RUNWAY_END'
    ,y='MSL_ALT'
    , color = "FLIGHT_ID"
    , labels = {'DISTANCE_FROM_RUNWAY_END':'Distance from runway end (feet)',
                'MSL_ALT':'Altitude above mean sea level (feet)',
                'APT_AIRCRAFT_RUNWAY_STAGE.1': 'Airport / Aicraft Type / Runway / Stage Length ID'
               }
)

file = fig_px.to_html()
with open(filepath+'\plot_high_quality_by_flight.html', 'w') as f:
    f.write(file)
    
fig_px.show()